Includes a function that loads images and their presaved bounding boxes and trains a landmark localisation method. 

Additionally, it includes code for verifying that the newly trained model performs 'reasonably' well on a validation set.

In [ ]:
from os.path import isdir, join, sep
from os import listdir
import numpy as np
from warnings import warn
from random import shuffle

import menpo.io as mio
import dlib
from menpofit.visualize import print_progress
import random

try:
    %matplotlib inline
except NameError:
    pass

Below are:

    a) the base path of the training images, 
    
    b) the name of the detection method (which is utilised to extract the bounding boxes), 
    
    c) the name of the pickle folder (where the trained model will be exported).

In [ ]:
detection_method = 'ramanan'
detection_glob = detection_method + '_*'

p0 = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/alignment_training/'
det_p = join('/vol/atlas/homes/grigoris/misc/2016_ijcv/data/alignment_training_bb/', detection_method, '')
assert(isdir(p0) and isdir(det_p))
path_pickles = '/vol/atlas/homes/grigoris/misc/2016_ijcv/data/pickles/'
assert(isdir(path_pickles))

# Load training images

In [ ]:
def get_train_images(img_p, det_p, ignore=False, detection_method=''):
    """
    Loads the training images and the respective bounding box + grounthtruth points for each image.
    The path with the landmark points is assumed to be the same as with the images and using the default menpo 
    behaviour, gt is loaded along with the image.
    :param img_p:   (string) Path with the images.
    :param det_p:   (string) Path with the bounding boxes. 
    :param ignore:  (bool, optional) Whether the images that have same bb as with gt should be loaded or not. 
                                     Only when detection_method != gt_bb.
    :param detection_method: (string) Name of the detection method.
    :return: (list) Loaded images for training.
    """
    assert(isdir(img_p) and isdir(det_p))
    assert(not (ignore and detection_method == 'gt_bb'))
    train_images = []
    ps = list(mio.image_paths(img_p))  # + '*_6.*'
    for imn in print_progress(ps):
        im = mio.import_image(imn)
        try:
            ln = mio.import_landmark_file(det_p + im.path.stem + '.pts')
        except ValueError:
            warn('The image {} is missing.'.format(im.path))
            continue
#             raise ValueError('The image {} is missing.'.format(im.path.stem))
        if not (ln.lms.n_points == 4):
            print(str(ln.path))
            continue
        im.landmarks['bb'] = ln
        if ignore:  # find which ones have the same bb as the gt.
            im.landmarks['gt'] = im.landmarks['PTS'].lms.bounding_box()
            if np.all(ln.lms.points == im.landmarks['gt'].lms.points):
                # ignore the images that have as "detector's bb" the gt bb
                continue
        im = im.crop_to_landmarks_proportion(0.3, 'bb')
        train_images.append(im)
    return train_images

In [ ]:
# names of different databases with static images that exist on p0. 
# The images from these databases will be loaded by calling get_train_images().
folds = ['ibug', 'afw', '300w', 'helen/trainset', 'helen/testset', 'lfpw/trainset', 'lfpw/testset']
folds = ['ibug', '300w', 'lfpw/trainset']  # , 'helen/testset'
images = []
for fold in folds:
    print(fold)
    im1 = get_train_images(p0 + fold + sep, det_p + fold + sep, ignore=True)
    assert(len(im1) > 50)
    images += im1
# shuffle(images)
random.Random(9).shuffle(images)
del im1

In [ ]:
# reduce the ammount of images
print('Images before reduction: {}'.format(len(images)))
images = images[:min(len(images), 2500)]
print(len(images))

In [ ]:
# from menpowidgets import visualize_images
# visualize_images(images)

# Train Ensemble Regression Trees (One millisecond Face Alignment paper)

In [ ]:
from menpofit.dlib import DlibERT

model = DlibERT(images, group='PTS', bounding_box_group_glob='bb*',
                scales=(1.0,), n_perturbations=0, n_dlib_perturbations=2, 
                n_iterations=14, verbose=True)

In [ ]:
# from menpowidgets import visualize_images
# visualize_images(images)

In [ ]:
p_save_model = path_pickles + 'modelln_' + detection_method + '_dlibERT.model'
# mio.export_pickle(model, p_save_model)

m = model.algorithms[0].dlib_model
m.save(p_save_model)

# SDM

In [ ]:
from menpofit.sdm import SupervisedDescentFitter
from menpofit.sdm.algorithm import ParametricShapeNewton
from functools import partial
from menpofit.fitter import noisy_shape_from_bounding_box
from menpo.feature import no_op, hellinger_vector_128_dsift, igo

n_pert = 4
n_iter = 4
patch_features=[hellinger_vector_128_dsift, hellinger_vector_128_dsift, hellinger_vector_128_dsift, no_op]
patch_shape=[(32, 32), (24, 24), (24, 24), (16, 16)]

ridge_alpha = 200
scale_perturb = 0.001
rotation_perturb = 0
translation_perturb = 0.05
better_bb = partial(noisy_shape_from_bounding_box, noise_percentage=[scale_perturb, 
                                                                     rotation_perturb, 
                                                                     translation_perturb])

print('Training Parametric Shape Ridge Regression SDM: Alpha {}'.format(ridge_alpha))

sdm = SupervisedDescentFitter(
    images,
    group='PTS', 
    bounding_box_group_glob='bb',
    reference_shape=None, 
    holistic_features=no_op, 
    sd_algorithm_cls=partial(ParametricShapeNewton, bias=True, alpha=ridge_alpha),
                             #shape_model_cls=smaller_ortho_pdm(0.98)),
#     patch_features=[partial(hellinger_vector_128_dsift, patch_shape=c) for c in [32, 24, 24, 16]],
    patch_features=patch_features,
    patch_shape=patch_shape,
    diagonal=200,
    scales=[0.5, 0.5, 1.0, 1.0],
    n_iterations=n_iter,
    n_perturbations=n_pert,
    perturb_from_gt_bounding_box=better_bb, 
    batch_size=None,
    verbose=True)

p_save_model = path_pickles + 'modelln_' + detection_method + '_sdm.pkl'
mio.export_pickle(sdm, p_save_model, overwrite=True)

In [ ]:
assert(0) # manual improvements in the sdm
from menpofit.sdm import SupervisedDescentFitter
from menpofit.sdm.algorithm import ParametricShapeNewton
from functools import partial
from menpofit.fitter import noisy_shape_from_bounding_box
from menpo.feature import no_op, hellinger_vector_128_dsift, igo

n_pert = 4
n_iter = 4
patch_features=[hellinger_vector_128_dsift, hellinger_vector_128_dsift, hellinger_vector_128_dsift, no_op]
patch_shape=[(32, 32), (24, 24), (24, 24), (16, 16)]

ridge_alpha = 200
scale_perturb = 0.001
rotation_perturb = 0
translation_perturb = 0.05
better_bb = partial(noisy_shape_from_bounding_box, noise_percentage=[scale_perturb, 
                                                                     rotation_perturb, 
                                                                     translation_perturb])

print('Training Parametric Shape Ridge Regression SDM: Alpha {}'.format(ridge_alpha))

sdm = SupervisedDescentFitter(
    images,
    group='PTS', 
    bounding_box_group_glob='bb',
    reference_shape=None, 
    holistic_features=no_op, 
    sd_algorithm_cls=partial(ParametricShapeNewton, bias=True, alpha=ridge_alpha),
                             #shape_model_cls=smaller_ortho_pdm(0.98)),
#     patch_features=[partial(hellinger_vector_128_dsift, patch_shape=c) for c in [32, 24, 24, 16]],
    patch_features=patch_features,
    patch_shape=patch_shape,
    diagonal=200,
    scales=[0.5, 0.5, 1.0, 1.0],
    n_iterations=n_iter,
    n_perturbations=n_pert,
    perturb_from_gt_bounding_box=better_bb, 
    batch_size=None,
    verbose=True)

p_save_model = path_pickles + 'sdm_valid/' + 'v1.2_modelln_' + detection_method 
#p_save_model += '_feats_' + patch_features[0].__name__ + '_patch_' 
p_save_model += '_patch_' 
p_save_model += str(patch_shape[0]) + '_' + str(patch_shape[1]) + '_perturb'
p_save_model += str(n_pert) + '_iter' + str(n_iter) + '_images' + str(len(images)) + '_noop_sdm.pkl'
mio.export_pickle(sdm, p_save_model, overwrite=True)

In [ ]:
from menpofit.sdm import RegularizedSDM
from menpo.feature import fast_dsift, igo, no_op

feature_pool = [[fast_dsift, no_op], [fast_dsift, fast_dsift], [no_op, no_op], [igo, no_op]]
patch_shape_pool = [[(12, 12), (18, 18)]]
pert_pool = [10, 15, 20]
iter_pool = [6, 10, 12]
scales = (.5, 1)
crop = 0.2
diagonal = 180

def train_sdm(images, features, patch_shape, n_pert, n_iter, cnt):
    fitter = RegularizedSDM(images, patch_features=features, verbose=True, 
                            diagonal=diagonal, n_perturbations=n_pert, group='PTS', 
                            patch_shape=patch_shape, alpha=100, n_iterations=n_iter, 
                            bounding_box_group_glob='bb*')
    p_save_model = path_pickles + 'sdm_valid/' + 'modelln_' + detection_method + '_cnt' + str(cnt)
    p_save_model += '_feats_' + features[0].__name__ + '_' + features[1].__name__ + '_patch_' 
    p_save_model += str(patch_shape[0]) + '_' + str(patch_shape[1]) + '_perturb'
    p_save_model += str(n_pert) + '_iter' + str(n_iter) + '_images' + str(len(images)) + '_sdm.pkl'
    mio.export_pickle(fitter, p_save_model, overwrite=True)

    
cnt = 0
for features in feature_pool:
    for patch_shape in patch_shape_pool:
        for n_pert in pert_pool:
            for n_iter in iter_pool:
                cnt += 1
                print('cnt: {}, patch shape: {}'.format(cnt, patch_shape))
                train_sdm(images, features, patch_shape, n_pert, n_iter, cnt)
        
    

# GN-DPM (patch aam) 

In [ ]:
# final one chosen from validation below
from menpo.feature import fast_dsift, igo, no_op
from menpofit.aam import PatchAAM

features = [fast_dsift, fast_dsift]
patch_shape = [(10, 10), (24, 24)]
scales = (.5, 1)
crop = 0.2
diagonal = 180


aam = PatchAAM(images, verbose=True, holistic_features=features, patch_shape=patch_shape,
               diagonal=diagonal, scales=scales, group='PTS')
p_save_model = path_pickles + 'modelln_' + detection_method
mio.export_pickle(aam, p_save_model + '_aam.pkl')

In [ ]:
# validation with two levels pyramid
from menpo.feature import fast_dsift, igo, no_op
from menpofit.aam import PatchAAM

p_save_model = path_pickles + 'aam_valid/modelln_' + detection_method
feature_pool = [[fast_dsift, no_op], [fast_dsift, fast_dsift]]
patch_shape_pool = [[(10, 10), (20, 20)], [(20, 20), (20, 20)], [(10, 10), (24, 24)], [(20, 20), (24, 24)]]
scales = (.5, 1)
crop = 0.2
diagonal = 180

cnt = 0
for features in feature_pool:
    for patch_shape in patch_shape_pool:
        cnt = cnt + 1
        print('cnt: {}, features: {}, patch shape: {}'.format(cnt, features, patch_shape))
        aam = PatchAAM(images, verbose=True, holistic_features=features, patch_shape=patch_shape,
                       diagonal=diagonal, scales=scales, group='PTS', batch_size=6000)
        ns = p_save_model + '_aam_' + str(cnt) + '_feats_' + features[0].__name__
        ns += features[1].__name__  + '_patch_' + str(patch_shape[0]) + '_'
        ns += str(patch_shape[1]) + '_images' + str(len(images)) + '_aam.pkl'
        aam.features = None
        try:
            mio.export_pickle(aam, ns, overwrite=True)
        except:
            import pickle
            with open(ns, 'wb') as f:
                pickle.dump(aam, f, protocol=4)    

In [ ]:
# validation with three levels pyramid
from menpo.feature import fast_dsift, igo, no_op, hellinger_vector_128_dsift
from menpofit.aam import PatchAAM

p_save_model = path_pickles + 'aam_valid/modelln_' + detection_method
feature_pool = [[fast_dsift, fast_dsift, no_op]]
patch_shape_pool = [[(12, 12), (20, 20), (16, 16)], [(10, 10), (20, 20), (20, 20)], 
                   [(10, 10), (24, 24), (24, 24)]]
scales = (.5, 1, 1)
crop = 0.2
diagonal = 180

cnt = 0
for features in feature_pool:
    for patch_shape in patch_shape_pool:
        cnt = cnt + 1
        print('cnt: {}, features: {}, patch shape: {}'.format(cnt, features, patch_shape))
        aam = PatchAAM(images, verbose=True, holistic_features=features, patch_shape=patch_shape,
                       diagonal=diagonal, scales=scales, group='PTS')
        ns = p_save_model + '_aam_' + str(cnt) + '_feats_' + features[0].__name__
        ns += features[1].__name__ + features[2].__name__ + '_patch_' + str(patch_shape[0]) + '_'
        ns += str(patch_shape[1]) + str(patch_shape[1]) + '_images' + str(len(images)) + '_aam.pkl'
        aam.features = None
        try:
            mio.export_pickle(aam, ns, overwrite=True)
        except:
            import pickle
            with open(ns, 'wb') as f:
                pickle.dump(aam, f, protocol=4)    

In [ ]:
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector


def return_detector(detection):
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        from functools import partial
        det = load_opencv_frontal_face_detector()
        return partial(det, min_neighbours=3)
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))
        
detector = return_detector(detection_method)

im = mio.import_builtin_asset.breakingbad_jpg()
del im.landmarks['PTS']
detector(im, group_prefix='bb')

# im.view_landmarks()
ll = model.fit_from_bb(im, im.landmarks['bb_0'].lms, max_iters=40)
im2 = ll.fitted_image
im2 = im2.crop_to_landmarks_proportion(0.3, group='final')
im2.view_landmarks(group='final')


In [ ]:
# im = mio.import_builtin_asset.breakingbad_jpg()
# im = im.crop_to_landmarks_proportion(0.3)
# random_rotations = np.random.randint(-20, 20, size=12)
# rotated_images = [im.rotate_ccw_about_centre(r) for r in random_rotations]
# for im_r in rotated_images:
#     detector(im_r)
#     try:
#         ll = model.fit_from_bb(im, im.landmarks['dlib_0'].lms, max_iters=40)
#     except KeyError:
#         im_r.view_landmarks(group='PTS', new_figure=True)
#         continue
#     im2 = ll.fitted_image
#     im2 = im2.crop_to_landmarks_proportion(0.3, group='final')
#     im2.view_landmarks(group='final', new_figure=True)
# #     im_r.view_landmarks(group='PTS', new_figure=True)
    

# Test the new model with new images

Testing the model trained above. Using a database i made, it checks that the fittings in unseen images make sense. There are asserts for mean error per image and the total error. 

In [ ]:
from os.path import isdir, join, isfile
from os import listdir
import numpy as np
import menpo.io as mio
from dlib import shape_predictor
from menpo.shape import PointCloud
from menpo.landmark import LandmarkGroup
from menpodetect import load_dlib_frontal_face_detector
from menpodetect import load_opencv_frontal_face_detector
from menpodetect.ffld2 import load_ffld2_frontal_face_detector
from menpodetect.dlib.conversion import pointgraph_to_rect
from menpofit.result import compute_normalise_point_to_point_error as comp_err
from menpofit.visualize import print_progress

def detection_to_pointgraph(detection):
    return PointCloud(np.array([(p.y, p.x) for p in detection.parts()]))

def compute_normalise_point_to_point_error_68(shape, gt_shape):
    normalizer = np.linalg.norm(gt_shape.points[36, :] - gt_shape.points[45, :])
    return comp_err(shape.points, gt_shape.points) / normalizer

def return_detector(detection):
    if detection == 'dlib':
        return load_dlib_frontal_face_detector()
    elif detection == 'opencv':
        from functools import partial
        det = load_opencv_frontal_face_detector()
        return partial(det, min_neighbours=3)
    elif detection == 'ffld2':
        return load_ffld2_frontal_face_detector()
    else:
        raise RuntimeError('Not a valid choice of detection ({}).'.format(detection))


In [ ]:
p0 = '/vol/atlas/homes/grigoris/Databases/personal/11_2015_test_localisation/'
method_landm_loc = 'dlibERT'
p_model = path_pickles + 'modelln_' + detection_method + '_dlibERT.model'
p_model = path_pickles + 'modelln_' + 'gt_bb' + '_dlibERT.model'
assert(isdir(p0) and isfile(p_model))
# detector = load_dlib_frontal_face_detector()
detector = return_detector(detection_method)

if method_landm_loc == 'dlibERT':
    predictor_dlib = shape_predictor(p_model)

images = list(mio.import_images(p0))
errors = []
for im in print_progress(images):
    im = im.crop_to_landmarks_proportion(0.3)
    detector(im, group_prefix='bb')
    l = list(im.landmarks.keys_matching('bb_*'))
    if len(l) == 0:
        print('No detection in {}.'.format(im.path.stem))
        continue
    assert(len(l) == 1)
    ln = im.landmarks['bb_0']
    if method_landm_loc == 'dlibERT':
        im_pili = np.array(im.as_PILImage())
        det_frame = predictor_dlib(im_pili, pointgraph_to_rect(ln.lms))
        init_pc = detection_to_pointgraph(det_frame)
    else:
        ft = model.fit_from_bb(im, ln.lms)
        init_pc = ft.final_shape
    ln1 = LandmarkGroup.init_with_all_label(init_pc)
    err = compute_normalise_point_to_point_error_68(ln1.lms, im.landmarks['PTS'].lms)
    assert(err < 0.3)
    errors.append(err)
mean_err = np.mean(np.array(errors))
assert(mean_err < 0.2)
print('Successfully predicted with mean error {}.'.format(mean_err))